# Visualização de Dados Geoespaciais

Se você já fez layouts no ArcGIS ou QGIS, aqui vamos aprender a fazer o mesmo programaticamente! É como ter um cartógrafo digital que segue suas instruções para criar mapas bonitos e informativos.

## Objetivos
- Criar mapas estáticos (como layouts tradicionais)
- Desenvolver mapas interativos (como WebGIS)
- Personalizar estilos (como design cartográfico)
- Combinar diferentes tipos de dados (como atlas digitais)

## 1. Configuração do Ambiente

Vamos preparar nosso estúdio de cartografia digital:

In [ ]:
# Nossa mesa de trabalho cartográfica:
import geopandas as gpd          # Para dados espaciais
import folium                    # Para mapas web (como Google Maps)
import matplotlib.pyplot as plt   # Para mapas estáticos (como layouts)
import contextily as ctx         # Para mapas base (como OpenStreetMap)
import numpy as np               # Para cálculos
import rasterio                  # Para dados matriciais
from rasterio.plot import show   # Para mostrar rasters
import seaborn as sns           # Para cores bonitas

# Configurando o tamanho padrão dos mapas
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Mapas Estáticos com Matplotlib e GeoPandas

Vamos começar com mapas tradicionais, como aqueles que você imprimiria:

In [ ]:
# Carregando nossa base cartográfica
estados = gpd.read_file('../data/raw/BR_UF_2022.shp')

# Criando diferentes tipos de mapas
# Como fazer vários layouts diferentes
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 15))

# Mapa básico
# Como um mapa político simples
estados.plot(ax=ax1)
ax1.set_title('Divisão Política')

# Mapa coroplético
# Como um mapa temático de áreas
estados.plot(column='area_km2', 
            cmap='YlOrRd',      # Esquema de cores
            legend=True,        # Legenda
            ax=ax2)
ax2.set_title('Área dos Estados')

# Mapa categórico
# Como um mapa de regiões
estados.plot(column='regiao',
            categorical=True,   # Cores por categoria
            legend=True,
            ax=ax3)
ax3.set_title('Regiões do Brasil')

# Mapa com rótulos
# Como adicionar nomes nos mapas
estados.plot(ax=ax4)
for idx, row in estados.iterrows():
    ax4.annotate(row['sigla'],             # Texto
                 xy=row.geometry.centroid.coords[0],  # Posição
                 ha='center')              # Alinhamento
ax4.set_title('Estados com Siglas')

plt.tight_layout()
plt.show()

## 3. Mapas Interativos com Folium

Agora vamos fazer mapas que você pode explorar como no Google Maps:

In [ ]:
# Criando um mapa base
# Como começar com um mapa do OpenStreetMap
m = folium.Map(location=[-15.7801, -47.9292],  # Brasília
               zoom_start=4)

# Adicionando camada coroplética
# Como adicionar um mapa temático interativo
folium.Choropleth(
    geo_data=estados.__geo_interface__,  # Geometrias
    name='choropleth',                   # Nome da camada
    data=estados,                        # Dados
    columns=['sigla', 'area_km2'],       # Colunas para o mapa
    key_on='feature.properties.sigla',   # Campo de ligação
    fill_color='YlOrRd',                 # Esquema de cores
    fill_opacity=0.7,                    # Transparência
    line_opacity=0.2,                    # Linhas
    legend_name='Área (km²)'             # Legenda
).add_to(m)

# Adicionando controle de camadas
# Como um TOC interativo
folium.LayerControl().add_to(m)

# Mostrando o mapa
m

## 4. Mapas com Contextily (Mapas Base)

Vamos adicionar mapas base bonitos aos nossos mapas estáticos:

In [ ]:
# Reprojetando para Web Mercator
# Como ajustar para o sistema do OpenStreetMap
estados_web = estados.to_crs(epsg=3857)

# Criando mapas com diferentes fundos
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))

# Mapa com OpenStreetMap
# Como usar um mapa topográfico de fundo
estados_web.plot(ax=ax1, alpha=0.5)
ctx.add_basemap(ax1)
ax1.set_title('Com OpenStreetMap')

# Mapa com relevo
# Como usar um mapa de terreno
estados_web.plot(ax=ax2, alpha=0.5)
ctx.add_basemap(ax2, source=ctx.providers.Stamen.Terrain)
ax2.set_title('Com Stamen Terrain')

plt.tight_layout()
plt.show()

## 5. Visualizações Temáticas Avançadas

Vamos criar mapas mais sofisticados para análises específicas:

In [ ]:
# Criando dados de exemplo
# Como simular dados demográficos
estados['populacao'] = np.random.randint(1000000, 50000000, size=len(estados))
estados['densidade'] = estados['populacao'] / estados['area_km2']

# Criando visualizações temáticas
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 15))

# Mapa de densidade
# Como um mapa demográfico
estados.plot(column='densidade',
            cmap='viridis',
            legend=True,
            legend_kwds={'label': 'Habitantes/km²'},
            ax=ax1)
ax1.set_title('Densidade Populacional')

# Mapa de bolhas
# Como símbolos proporcionais
estados.plot(ax=ax2)
estados.centroid.plot(ax=ax2,
                     markersize=estados['populacao']/500000,
                     alpha=0.5)
ax2.set_title('População (tamanho proporcional)')

# Mapa de calor
# Como uma superfície de densidade
estados.plot(column='densidade',
            cmap='hot_r',
            legend=True,
            ax=ax3)
ax3.set_title('Mapa de Calor - Densidade')

# Mapa categorizado
# Como classes de densidade
estados['categoria_densidade'] = pd.qcut(estados['densidade'], 
                                        q=5, 
                                        labels=['Muito Baixa', 'Baixa', 'Média', 
                                               'Alta', 'Muito Alta'])
estados.plot(column='categoria_densidade',
            categorical=True,
            legend=True,
            ax=ax4)
ax4.set_title('Categorias de Densidade')

plt.tight_layout()
plt.show()

## 6. Combinando Dados Vetoriais e Matriciais

Vamos criar mapas que misturam diferentes tipos de dados:

In [ ]:
# Lendo um raster
# Como carregar uma imagem ou MDT
with rasterio.open('../data/processed/exemplo.tif') as src:
    raster = src.read(1)
    raster_extent = [src.bounds.left, src.bounds.right,
                     src.bounds.bottom, src.bounds.top]

# Criando um mapa combinado
# Como sobrepor vetores em uma imagem
fig, ax = plt.subplots(figsize=(12, 8))

# Plotando o raster
# Como uma imagem de fundo
im = ax.imshow(raster, 
               extent=raster_extent,
               cmap='terrain')

# Plotando os vetores
# Como uma camada de overlay
estados.plot(ax=ax,
            facecolor='none',
            edgecolor='black',
            linewidth=1)

plt.colorbar(im, label='Valor')
plt.title('Limites sobre Raster')
plt.show()

## 7. Exercícios Práticos

Vamos praticar como um cartógrafo digital:

1. Mapas Temáticos:
   - Crie um mapa de dados socioeconômicos
   - Adicione elementos cartográficos
   - Escolha cores apropriadas
   - Exporte em alta resolução

2. Mapas Web:
   - Faça um mapa Folium multicamadas
   - Adicione popups informativos
   - Configure a legenda e controles
   - Personalize os estilos

3. Atlas Digital:
   - Combine vetores e rasters
   - Crie uma série de mapas temáticos
   - Adicione elementos cartográficos
   - Monte um layout profissional